# Algorithmus für künstliche Intelligenz mit CNN

## Libaray

In [1]:

import os
import glob
import time

import numpy as np
import random

import optuna

import plotly
import plotly.io as pio

import torch
import torch.optim as optim

from torch import nn

from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.utils.data import Dataset

import matplotlib.pyplot as plt


In [2]:

torch.set_float32_matmul_precision("high")


## Modell auf der GPU ausführen

In [3]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


## Modell Name

In [4]:

model_name = "1d_cnn_model_test"


## Einlesen der Daten

In [5]:

class Load_Data:
    
    def __init__(self, root_dir = ""):
        
        self.root_dir = root_dir
        
        self.signal_list_all = []
        self.lable_list_all = []
        
        self.signal_list_80 = []
        self.signal_list_20 = []
        self.lable_list_80 = []
        self.lable_list_20 = []
        
        self.load_data()
        self.sort_data()
        
    def load_data(self):
        
        for dirpath, dirnames, filenames in os.walk(self.root_dir):
            for filename in filenames:
                if filename.endswith(".npz"): 
                    
                    file_path = os.path.join(dirpath, filename)
                    data = np.load(file_path)

                    signal_ = data["signal"]
                    signal_full = signal_[np.newaxis] # An additional dimension is needed for later processing
                    signal = signal_full[:,::]  # Halving of the input data and thus of the features
                    
                    heart_rate = data["heart_uart"]
                    respiration_rate = data["respiration_radar"]
                    
                    heart_rate_t = heart_rate
                    respiration_rate_t = respiration_rate
                    signal_t = signal
   
                    self.lable_list_all.append([heart_rate_t, respiration_rate_t])
                    self.signal_list_all.append(signal_t)
    
    def sort_data(self):   
        
        lable = np.array(self.lable_list_all)
        signal = np.array(self.signal_list_all)

        unique_elements, counts_elements = np.unique(lable[:,0], return_counts=True)
        
        print(counts_elements)
        print(unique_elements)
        
        for element in unique_elements:
            
            total_count = counts_elements[np.where(unique_elements == element)]
            
            count_80 = int(total_count * 0.8)
            
            arr_l_80, arr_l_20 = np.split(lable[(lable[:,0] == element)], [count_80])
            arr_s_80, arr_s_20 = np.split(signal[(lable[:,0] == element)], [count_80])

            for count in range(len(arr_l_80)):
                self.lable_list_80.append(arr_l_80[count])
                
            for count in range(len(arr_l_20)):   
                self.lable_list_20.append(arr_l_20[count])
                
            for count in range(len(arr_l_80)):
                self.signal_list_80.append(arr_s_80[count])
                
            for count in range(len(arr_l_20)):   
                self.signal_list_20.append(arr_s_20[count])     
                
    def get_dataset_80(self):   
        return self.signal_list_80, self.lable_list_80
        
    def get_dataset_20(self):   
        return self.signal_list_20, self.lable_list_20
    

In [6]:

class MyDataset(Dataset):
    
    def __init__(self, signal_list, lable_list):

        self.lable_list_ = lable_list
        self.signal_list_ = signal_list
        self.lable_list = []
        self.signal_list = []
        
        self.append_data()

    def append_data(self):
        
        lable = np.array(self.lable_list_)
        signal = np.array(self.signal_list_)

        lable_ = torch.from_numpy(lable).float().to(device)
        signal_ = torch.from_numpy(signal).float().to(device)
        
        for value in lable_:
            self.lable_list.append(value)
            
        for value in signal_:
            self.signal_list.append(value)

    def __len__(self):
        

        return len(self.lable_list) 

    def __getitem__(self, idx):

        return self.signal_list[idx], self.lable_list[idx]
    

In [7]:

root_folder_train = "data"
dataset = Load_Data(root_folder_train)


[  4  49  25  27  53  48  34  66  86 144  96  90 122 199 231 234 257 209
 220 106 223 221 157 120 207 167 118 131  71  76 113  19  10   9   9   7
   2]
[56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79
 80 81 82 83 84 85 86 87 88 89 90 91 92]


In [8]:

signal_list_80, lable_list_80 = dataset.get_dataset_80()
dataset_train = MyDataset(signal_list_80, lable_list_80)


In [9]:

signal_list_20, lable_list_20 = dataset.get_dataset_20()
dataset_test = MyDataset(signal_list_20, lable_list_20)


In [10]:

dataloader_train = DataLoader(dataset_train, batch_size=32, shuffle=True)


In [11]:

for i_batch, (inputs, targets) in enumerate(dataloader_train):

    print(i_batch)
    print(inputs.shape)
    print(targets.shape)
    print(targets[:,0:1].shape)
    print("next")

    break
    

0
torch.Size([32, 1, 1500])
torch.Size([32, 2])
torch.Size([32, 1])
next


In [12]:

dataloader_test = DataLoader(dataset_test, batch_size=32, shuffle=True)


In [13]:

for i_batch, (inputs, targets) in enumerate(dataloader_test):

    print(i_batch)
    print(inputs.shape)
    print(targets.shape)
    print(targets[:,0:1].shape)
    print("next")

    break
    

0
torch.Size([32, 1, 1500])
torch.Size([32, 2])
torch.Size([32, 1])
next


## Einlesen der Daten

In [14]:

class Pulse_Respiration_CNN(nn.Module):
    
    def __init__(self, 
                 input_width, 
                 conv_layer_, 
                 fc_layers_, 
                 conv_channel_, 
                 conv_kernel_, 
                 pool_kernel_, 
                 pool_stride_,
                 fully_connected_):
        
        super(Pulse_Respiration_CNN, self).__init__()

        self.af_1 = nn.Tanh()
        self.af_2 = nn.ReLU()

        self.conv_1_layers = nn.ModuleList()
        self.bn_1_layers = nn.ModuleList()
        self.pool_1_layers = nn.ModuleList()
        
        self.fc_layers = nn.ModuleList()
        
        conv_layer = conv_layer_
        fc_layers = fc_layers_
        
        print(f"conv_layer: {conv_layer}")
        print(f"fc_layers: {fc_layers}")

        Width = input_width
        in_channels = 1

        for i in range(conv_layer):
            
            conv_channel = conv_channel_[i]
            conv_kernel = conv_kernel_[i]
            conv_stride = 1 
            
            pool_kernel = pool_kernel_[i]
            pool_stride = pool_stride_[i]
     
            Kernel = conv_kernel # kernel_size
            Padding = 0 # (Kernel - 1) // 2 # padding
            Stride = conv_stride # stride
            Width_a = ((Width - conv_kernel + 2 * Padding) // conv_stride) + 1
            
            Width_b = ((Width_a - pool_kernel) // pool_stride) + 1
            
            if (Width_b < 8):
                Kernel = conv_kernel + pool_kernel
            
                Padding = (Kernel - 1) // 2 # padding
                Width = ((Width - conv_kernel + 2 * Padding) // conv_stride) + 1
                Width = ((Width - pool_kernel) // pool_stride) + 1
            else:
                Width = Width_b
            
            self.conv_1_layers.append(nn.Conv1d(in_channels=in_channels, out_channels=conv_channel, kernel_size=conv_kernel, stride=conv_stride, padding=Padding))
            in_channels = conv_channel
            
            self.bn_1_layers.append(nn.BatchNorm1d(conv_channel))

            self.pool_1_layers.append(nn.MaxPool1d(pool_kernel, pool_stride)) 
            
            
            print(f"s{i}_conv_channel_1: {conv_channel} s{i}_conv_kernel_1: {conv_kernel} s{i}_conv_stride_1: {conv_stride} s{i}_padding_1: {Padding} s{i}_pool_kernel_1: {pool_kernel} s{i}_pool_stride_1: {pool_stride} s{i}_Width_1: {Width}")
        
        out_features = Width * in_channels
        
        print(f"s{i}_features: {out_features} = Width: {Width} * in_channels: {in_channels}")

        for i in range(fc_layers):
            
            out_features_next = fully_connected_[i]
            
            print(f"s{i}_fully_connected_: {out_features_next}")
            
            self.fc_layers.append(nn.Linear(in_features=out_features, out_features=out_features_next))
            out_features = out_features_next
            
        self.fc_out = nn.Linear(in_features=out_features, out_features=1)    
        
    def forward(self, x):

        for conv_1, bn_1, pool_1 in zip(self.conv_1_layers, self.bn_1_layers, self.pool_1_layers):
            
            x = conv_1(x)
            x = bn_1(x)
            x = self.af_1(x)
            x = pool_1(x)

        x = x.view(x.size(0), -1)
        
        for layer_1 in self.fc_layers:
            
            x = layer_1(x)
            x = self.af_2(x)
            
        x = self.fc_out(x)
        
        return x
        

## Modellinstanz erstellen

### Parameter 

##### conv_layer: 8
##### fc_layers: 2
##### s0_conv_channel_1: 16 s0_conv_kernel_1: 7 s0_conv_stride_1: 1 s0_padding_1: 0 s0_pool_kernel_1: 5 s0_pool_stride_1: 2 s0_Width_1: 745
##### s1_conv_channel_1: 16 s1_conv_kernel_1: 29 s1_conv_stride_1: 1 s1_padding_1: 0 s1_pool_kernel_1: 5 s1_pool_stride_1: 1 s1_Width_1: 713
##### s2_conv_channel_1: 16 s2_conv_kernel_1: 3 s2_conv_stride_1: 1 s2_padding_1: 0 s2_pool_kernel_1: 3 s2_pool_stride_1: 1 s2_Width_1: 709
##### s3_conv_channel_1: 32 s3_conv_kernel_1: 29 s3_conv_stride_1: 1 s3_padding_1: 0 s3_pool_kernel_1: 4 s3_pool_stride_1: 1 s3_Width_1: 678
##### s4_conv_channel_1: 16 s4_conv_kernel_1: 3 s4_conv_stride_1: 1 s4_padding_1: 0 s4_pool_kernel_1: 4 s4_pool_stride_1: 1 s4_Width_1: 673
##### s5_conv_channel_1: 64 s5_conv_kernel_1: 29 s5_conv_stride_1: 1 s5_padding_1: 0 s5_pool_kernel_1: 3 s5_pool_stride_1: 1 s5_Width_1: 643
##### s6_conv_channel_1: 32 s6_conv_kernel_1: 7 s6_conv_stride_1: 1 s6_padding_1: 0 s6_pool_kernel_1: 4 s6_pool_stride_1: 1 s6_Width_1: 634
##### s7_conv_channel_1: 16 s7_conv_kernel_1: 17 s7_conv_stride_1: 1 s7_padding_1: 0 s7_pool_kernel_1: 5 s7_pool_stride_1: 1 s7_Width_1: 614
##### s7_features: 9824 = Width: 614 * in_channels: 16
##### s0_fully_connected_: 32
##### s1_fully_connected_: 32


In [33]:

def train_model():
    
    start_time = time.time()

    for i_batch, (inputs, targets) in enumerate(dataloader_train):
        input_s_ = inputs.shape[2]
        
    conv_layer = 8
    fc_layers = 2
    
    conv_channel = [16,16,16,32,16,64,32,16]
    conv_kernel = [5,29,3,29,3,29,7,17]
    pool_kernel = [5,5,3,4,4,3,4,5]
    pool_stride = [2,1,1,1,1,1,1,1]
    
    fully_connected = [32,32]

    model = Pulse_Respiration_CNN(input_width = input_s_, 
                                  conv_layer_ = conv_layer, 
                                  fc_layers_ = fc_layers, 
                                  conv_channel_ = conv_channel, 
                                  conv_kernel_ = conv_kernel, 
                                  pool_kernel_ = pool_kernel, 
                                  pool_stride_ = pool_stride,
                                  fully_connected_ = fully_connected).to(device)

    dateipfad = model_name
    
    if os.path.isfile(dateipfad):
        print("Die Datei existiert.")
        model.load_state_dict(torch.load(model_name))
        
    
    learning_rate = 0.001
    L2_regularization = 1e-5 # recommended = 1e-5 # default = 0

    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=L2_regularization) 
    criterion = nn.MSELoss()
    
    epochs = 1000

    for epoch in range(epochs):

        model.train()
        train_loss = 0.0

        for i_batch, (inputs, targets) in enumerate(dataloader_train):

            optimizer.zero_grad() 
            outputs = model(inputs) 
            loss = criterion(outputs, targets[:,0:1]) # breathing and heart
            loss.backward() 
            optimizer.step() 

            train_loss += loss.item()
            
        train_loss = train_loss /len(dataloader_train)
            
        model.eval() 
        valid_loss = 0.0

        with torch.no_grad():
            
            for i_batch, (inputs, targets) in enumerate(dataloader_test):

                outputs = model(inputs) 
                loss = criterion(outputs, targets[:,0:1]) 
                valid_loss += loss.item()

            valid_loss = valid_loss /len(dataloader_test)

        if (epoch + 3) % 100 == 0:
            print(f"Epoch [{epoch + 3}/{epochs}], Run Loss: {train_loss:.4f}, Val Loss: {valid_loss:.4f}")
        
        if (epoch + 2) % 1000 == 0:
            print(f"predictions [{outputs.cpu().numpy()[0]}], targets: {targets[:,0:1].cpu().numpy()[0]}")

        if (epoch + 1) % 1000 == 0:
            torch.save(model.state_dict(), model_name + ".pt")
    
    end_time = time.time()

    execution_time = end_time - start_time
    
    print("Die Ausführungszeit beträgt: ", execution_time, " Sekunden")
    
    return valid_loss


In [34]:

valid_loss = train_model()


conv_layer: 8
fc_layers: 2
s0_conv_channel_1: 32 s0_conv_kernel_1: 3 s0_conv_stride_1: 1 s0_padding_1: 0 s0_pool_kernel_1: 3 s0_pool_stride_1: 1 s0_Width_1: 1496
s1_conv_channel_1: 16 s1_conv_kernel_1: 5 s1_conv_stride_1: 1 s1_padding_1: 0 s1_pool_kernel_1: 3 s1_pool_stride_1: 2 s1_Width_1: 745
s2_conv_channel_1: 32 s2_conv_kernel_1: 17 s2_conv_stride_1: 1 s2_padding_1: 0 s2_pool_kernel_1: 3 s2_pool_stride_1: 3 s2_Width_1: 243
s3_conv_channel_1: 16 s3_conv_kernel_1: 29 s3_conv_stride_1: 1 s3_padding_1: 0 s3_pool_kernel_1: 4 s3_pool_stride_1: 1 s3_Width_1: 212
s4_conv_channel_1: 32 s4_conv_kernel_1: 3 s4_conv_stride_1: 1 s4_padding_1: 0 s4_pool_kernel_1: 4 s4_pool_stride_1: 2 s4_Width_1: 104
s5_conv_channel_1: 16 s5_conv_kernel_1: 5 s5_conv_stride_1: 1 s5_padding_1: 0 s5_pool_kernel_1: 4 s5_pool_stride_1: 3 s5_Width_1: 33
s6_conv_channel_1: 32 s6_conv_kernel_1: 17 s6_conv_stride_1: 1 s6_padding_1: 0 s6_pool_kernel_1: 5 s6_pool_stride_1: 1 s6_Width_1: 13
s7_conv_channel_1: 16 s7_conv_ker

In [ ]:

print(valid_loss)
